In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"]
os.environ["WATSONX_APIKEY"] 
os.environ["PINECONE_API_KEY"]


'pcsk_vpL3C_NXGzXyQowN4QmyvjLNMpAUX47RGcMvyXh8wLMyMjWTdD2SHWdhpMApmMg18Xma9'

In [3]:
from langchain_ibm import ChatWatsonx

model = ChatWatsonx(
    model_id="meta-llama/llama-3-1-8b-instruct",
    url="https://eu-de.ml.cloud.ibm.com",
    project_id="aeabba3a-e8d1-4181-bcff-d0a1b639e896"
)

/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot/.conda/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'meta-llama/llama-3-1-8b-instruct' is in deprecated state from 2025-01-22 until 2025-05-30. IDs of alternative models: llama-3-2-11b-vision-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [4]:
# from langchain_ibm import WatsonxEmbeddings

# embeddings = WatsonxEmbeddings(
#     model_id="ibm/slate-125m-english-rtrvr",
#     url="https://eu-de.ml.cloud.ibm.com",
#     project_id="aeabba3a-e8d1-4181-bcff-d0a1b639e896",
# )

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-large")

/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vectors = embeddings.embed_query("How are you")
len(vectors)

768

In [6]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], 
              environment="us-east-1")
index_name = "tal-chatbot"
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

Loading Documents


In [7]:
from langchain_community.document_loaders import JSONLoader
file_path = "/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot/DataPrep/converters.json"

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.[]',  # This will iterate over each item
    text_content=False  # We'll use the entire object as content
)

documents = loader.load()


In [8]:
len(documents)

65

Uploading Embeddings


In [9]:
vectorstore = PineconeVectorStore.from_documents(
    documents,
    embedding=embeddings,
    index_name=index_name
)

Consine Sim Retrieval

In [19]:
from langchain_core.prompts import ChatPromptTemplate

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a technical specification assistant. For any query about components:
        1. List ALL matching items from context
        2. Include full technical details
        3. Present in markdown tables
        4. Never omit specifications"""),
            ("human", """Context: {context}
            
        Question: {question}

        Answer:""")
])

In [20]:
from langchain import hub
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"], k=100)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    # messages = prompt.invoke({"question": state["question"], "context": docs_content})
    # response = model.invoke(messages)
    # return {"answer": response.content}
    messages = custom_prompt.invoke({
        "question": state["question"],
        "context": docs_content
    })
    
    response = model.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [24]:
response = graph.invoke({"question": "give me info on convertor 930562"})
print(response["answer"])

| **ARTNR** | **CONVERTER DESCRIPTION** | **DIMLIST TYPE** | **STRAIN RELIEF** | **LOCATION** | **DIMMABILITY** | **CCR (AMPLITUDE)** | **SIZE: L*B*H (mm)** | **EFFICIENCY @full load** | **IP** | **CLASS** | **NOM. INPUT VOLTAGE (V)** | **OUTPUT VOLTAGE (V)** | **lamps** |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 930562 | POWERLED CONVERTER REMOTE 350mA 13,5W IP20 MAINS DIM LC 10% | dimlist R | YES | INDOOR | MAINS DIM LC 10% | YES | 109*53*25 | 0.84 | 20 | 2 | 198 - 264 | 18 - 38 | Single led XPE, Thinksmall/floorspot WC luxeon MX, *MIX 6 monocolor, Cedrus quantum, *MIX 6 halosphere, MIX 13 halosphere, ORBITAL monocolor, ORBITAL halosphere, Beaufort\u00b2, Haloled, B4, MIX 26 monocolor, *BOA WC |
